In [1]:
# Supported values are: 'code_interpreter', 'function', and 'file_search'."라는 오류가 발생합니다. 
# retrieval를 file_search로 변경한 예제입니다

import os
os.environ["OPENAI_API_KEY"] = "sk-"
from openai import OpenAI
 
client = OpenAI()

In [2]:
import json
def show_json(obj):
  display(json.loads(obj.model_dump_json()))
    
assistant = client.beta.assistants.create(
  name="smartfarm",
  instructions="당신은 스마트팜에 대한 전문가입니다. pdf 파일에 기반하여 답변해주세요.",
  model="gpt-4o",
  tools=[{"type": "file_search"}]
)
show_json(assistant)

{'id': 'asst_w0dtLv4QDtBdYhYTtSQEx7A7',
 'created_at': 1720762344,
 'description': None,
 'instructions': '당신은 스마트팜에 대한 전문가입니다. pdf 파일에 기반하여 답변해주세요.',
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'smartfarm',
 'object': 'assistant',
 'tools': [{'type': 'file_search', 'file_search': None}],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None,
  'file_search': {'vector_store_ids': []}},
 'top_p': 1.0}

In [3]:
vector_store = client.beta.vector_stores.create(
    name="file_store"
)

# 업로드할 파일 PDF 준비
file_paths = ["d:/스마트팜.pdf"]
file_streams = [open(path, "rb") for path in file_paths]

# 파일을 업로드하고, vector store에 추가
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id = vector_store.id, files=file_streams,
)

# 파일의 업로드 상태 확인
print(file_batch.status)
print(file_batch.file_counts)

show_json(vector_store)

failed
FileCounts(cancelled=0, completed=0, failed=1, in_progress=0, total=1)


{'id': 'vs_DzoGZpMqZnxyMBkPWcERU9ne',
 'created_at': 1720762344,
 'file_counts': {'cancelled': 0,
  'completed': 0,
  'failed': 0,
  'in_progress': 0,
  'total': 0},
 'last_active_at': 1720762344,
 'metadata': {},
 'name': 'file_store',
 'object': 'vector_store',
 'status': 'completed',
 'usage_bytes': 0,
 'expires_after': None,
 'expires_at': None}

In [4]:
# 생성한 vector store를 참조할 수 있도록 업데이트
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)
show_json(assistant)

{'id': 'asst_w0dtLv4QDtBdYhYTtSQEx7A7',
 'created_at': 1720762344,
 'description': None,
 'instructions': '당신은 스마트팜에 대한 전문가입니다. pdf 파일에 기반하여 답변해주세요.',
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'smartfarm',
 'object': 'assistant',
 'tools': [{'type': 'file_search', 'file_search': None}],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None,
  'file_search': {'vector_store_ids': ['vs_DzoGZpMqZnxyMBkPWcERU9ne']}},
 'top_p': 1.0}

In [5]:
# thread, message, run 생성 및 실행
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content= "스마트팜이란?",
)

run = client.beta.threads.runs.create_and_poll(
    thread_id = thread.id,
    assistant_id = assistant.id,
)
show_json(run)

{'id': 'run_2Zm1Y9aGr3AFJoCb4CPPUgiw',
 'assistant_id': 'asst_w0dtLv4QDtBdYhYTtSQEx7A7',
 'cancelled_at': None,
 'completed_at': 1720762355,
 'created_at': 1720762349,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '당신은 스마트팜에 대한 전문가입니다. pdf 파일에 기반하여 답변해주세요.',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1720762349,
 'status': 'completed',
 'thread_id': 'thread_wiJMH0I5svWufT8xxrXgVkrh',
 'tool_choice': 'auto',
 'tools': [{'type': 'file_search', 'file_search': None}],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': {'completion_tokens': 326,
  'prompt_tokens': 1573,
  'total_tokens': 1899},
 'temperature': 1.0,
 'top_p': 1.0,
 'tool_resources': {}}

In [6]:
# GPT-4o 답변 출력
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  show_json(messages)
  #print_message(thread.id)
else:
  print(run.status)

{'data': [{'id': 'msg_NNzxlayO0fK7GfzFMyluZkTq',
   'assistant_id': 'asst_w0dtLv4QDtBdYhYTtSQEx7A7',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': '스마트팜(Smart Farm)이란, 정보통신기술(ICT, Information and Communication Technology)을 농축산업에 융합하여, 작물과 가축의 생육 및 환경 데이터를 실시간으로 모니터링하고 분석하여 최적의 생육 환경을 유지함으로써 생산성과 효율성을 극대화하는 농업 시스템을 말합니다. 스마트팜 시스템은 다음과 같은 요소들을 포함합니다:\n\n1. **센서 및 IoT 장치**: 토양 습도, 온도, 조도 등 환경 데이터를 수집하기 위해 다양한 센서와 IoT(Internet of Things) 장치를 활용합니다.\n2. **빅데이터 분석**: 수집된 데이터를 분석하여 작물 생육 상태를 파악하고, 필요한 조치를 예측하며, 최적의 농업 방식을 제안합니다.\n3. **자동화 제어 시스템**: 온실이나 재배 시설에서 환경을 자동으로 조절하는 시스템을 통해 온도, 습도, 환기 등을 자동으로 관리합니다.\n4. **모바일 및 웹 애플리케이션**: 농업인들이 언제 어디서나 스마트폰이나 컴퓨터를 통해 농장의 상태를 확인하고 조치를 취할 수 있도록 지원합니다.\n\n이를 통해 농업 생산성을 높이고 자원 사용을 최적화하며, 기후 변화나 해충 등 외부 요인에 대한 대응력을 강화할 수 있습니다.'},
     'type': 'text'}],
   'created_at': 1720762351,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role'